In [2]:
import socket
from struct import *
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline
import time
import numpy as np
import pandas as pd
import scipy.constants as spconst
from sklearn.externals import joblib
from IPython.display import clear_output

In [3]:
def get_rotation_matrix(data):
    #print(unpack_from('!f',data,36)[0])
    orientation_z = np.radians(unpack_from('!f',data,36)[0])
    c, s = np.cos(orientation_z), np.sin(orientation_z)
    Rz = np.array(((c,-s,0),(s,c,0), (0,0,1)))
    
    orientation_x = np.radians(unpack_from('!f',data,40)[0])
    c, s = np.cos(orientation_x), np.sin(orientation_x)
    Rx = np.array(((1,0,0),(0,c,-s), (0,s,c)))
    
    orientation_y = np.radians(unpack_from('!f',data,44)[0])
    c, s = np.cos(orientation_y), np.sin(orientation_y)
    Ry = np.array(((c,0,s),(0,1,0),(-s,0,c)))
    
    #print("RX:",Rx)
    #print("Ry:",Ry)
    orientation = np.array([orientation_x,orientation_y,orientation_z])
    #print(orientation)
    return Rz.dot(Ry).dot(Rx)

In [4]:
def get_accelerometer(data):
    lin_acc_x = unpack_from('!f',data,0)
    lin_acc_y = unpack_from('!f',data,4)
    lin_acc_z = unpack_from('!f',data,8)
    lin_acc = np.array([lin_acc_x,lin_acc_y,lin_acc_z])
    
    grav_x = unpack_from('!f',data,12)
    grav_y = unpack_from('!f',data,16)
    grav_z = unpack_from('!f',data,20)
    grav = np.array([grav_x,grav_y,grav_z])
    
    acc = lin_acc + (grav * spconst.g)
    return acc

In [5]:
def get_transformed_accelerometer(data):
    acc = get_accelerometer(data)
    R = get_rotation_matrix(data)
    #print(orientation)
    return R.dot(acc)
    #return acc

In [6]:
#UDP_IP = "96.49.100.238"
#UDP_IP = "127.0.0.1"
UDP_IP = socket.gethostbyname(socket.gethostname())
print ("Receiver IP: ", UDP_IP)
#UDP_PORT = 6000
UDP_PORT = int(input ("Enter Port "))
print ("Port: ", UDP_PORT)
sock = socket.socket(socket.AF_INET, # Internet
                    socket.SOCK_DGRAM) # UDP
sock.bind((UDP_IP, UDP_PORT))
model = joblib.load('activity_classifier_acc.pkl')
sensor_history = pd.DataFrame()
while True:
    data, addr = sock.recvfrom(1024) # buffer size is 1024 bytes
    acc = get_transformed_accelerometer(data)
    #print(acc)
    #get std features
    if len(sensor_history) == 0:
        sensor_history = pd.DataFrame(acc.T, columns=['x','y','z'])
    else:
        sensor_history_new = pd.DataFrame(acc.T, columns=['x','y','z'])
        sensor_history = sensor_history.append(sensor_history_new)
        if len(sensor_history) == 35:
            sensor_history_new['std_x'] = sensor_history['x'].std()
            sensor_history_new['std_y'] = sensor_history['y'].std()
            sensor_history_new['std_z'] = sensor_history['z'].std()
#             sensor_history_new['mean_x'] = sensor_history['x'].mean()
#             sensor_history_new['mean_y'] = sensor_history['y'].mean()
#             sensor_history_new['mean_z'] = sensor_history['z'].mean()
            clear_output(wait=True)
           # print("pre transformation acc:", get_accelerometer(data))
            #print("post transformation acc:",acc)
            #print(sensor_history_new,"\n", model.predict(sensor_history_new))
            print(model.predict(sensor_history_new))
            sensor_history = sensor_history.iloc[1:]
        else:
            clear_output(wait=True)
            print(len(sensor_history))
    

['sit']


KeyboardInterrupt: 